In [ ]:
# Import packages to visualize the classifer
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# Import packages to do the classifying
import numpy as np
from sklearn.svm import SVC

from DecisionPlot import plot_decision_surface, plot_samples
from HyperplaneMarginPlot import plot_hyperplane_margin

# Support Vector Machines

Wir erzeugen uns einen Datensatz, der alle Punkte im oberen rechten und unteren linken Quadranten mit "1" labeled und alle 
anderen Punkte mit "-1":

In [ ]:
# Create Dataset
np.random.seed(0)
X_xor = np.random.randn(200, 2)
y_xor = np.logical_xor(X_xor[:, 0] > 0,
                       X_xor[:, 1] > 0)
y_xor = np.where(y_xor, -1, 1)

ax = plot_samples(X_xor, y_xor)
ax.axvline(0, c='grey', linestyle='-.', linewidth=1)
ax.axhline(0, c='grey', linestyle='-.', linewidth=1);

In [ ]:
# Create a SVC classifier using a linear kernel
C = 1000.0

svm = SVC(kernel='linear', C=C, random_state=0)
# Train the classifier
svm.fit(X_xor, y_xor)

ax = plot_hyperplane_margin(X_xor, y_xor, svm)
ax.set_title(f'Hyperebene und Margins für SVM mit Linearem Kernel, C={C}');

In [ ]:
# Visualize the decision boundaries
ax = plot_decision_surface(X_xor, y_xor, svm);
ax.set_title(f'SVM mit Linearem Kernel, C={C}')
print(f'Anzahl Support Vectors: {svm.n_support_}')


## RBF Kernel

Beim Training einer SVM mit dem Kernel der Radialen Basisfunktion (RBF) müssen zwei Parameter berücksichtigt werden: C und $\gamma$. 

Der Parameter C, der allen SVM-Kerneln gemeinsam ist, wägt eine falsche Klassifizierung von Trainingsbeispielen gegen eine einfachere Entscheidungsoberfläche ab. Ein niedriges C macht die Entscheidungsoberfläche glatt, während ein hohes C darauf abzielt, alle Trainingsbeispiele richtig zu klassifizieren. C wirkt insofern auf die gesamte **Trainingsmenge**.

$\gamma$ definiert, wie viel Einfluss ein **einzelnes** Trainingsbeispiel hat. Je größer $\gamma$ ist, desto näher müssen andere Samples sein, um beeinflusst zu werden.

Das Verhalten des Modells ist sehr empfindlich gegenüber dem $\gamma$-Parameter. Wenn $\gamma$ zu groß ist, schließt der Radius des Einflussbereichs der Stützvektoren nur den Stützvektor selbst ein, und kein Betrag der Regularisierung mit C wird in der Lage sein, eine Überanpassung zu verhindern.

Wenn $\gamma$ sehr klein ist, ist das Modell zu stark eingeschränkt und kann die Komplexität oder "Form" der Daten nicht erfassen. Der Einflussbereich eines ausgewählten Stützvektors würde das gesamte Trainingsset umfassen. Das resultierende Modell verhält sich dann ähnlich wie ein lineares Modell.

Die richtige Wahl von C und $\gamma$ ist entscheidend für die Leistung der SVM.

In [ ]:
# Create a SVC classifier using an RBF kernel
def pltSvmRbf(C, gamma):
    svm = SVC(kernel='rbf', random_state=0, gamma=gamma, C=C)
    svm.fit(X_xor, y_xor)

    ax = plot_decision_surface(X_xor, y_xor, svm);
    plot_hyperplane_margin(X_xor, y_xor, svm, ax)
    ax.set_title(r'SVM mit RBF Kernel ($e^{-\gamma{\parallel x-x´\parallel}^2}$), $\gamma$=' + f'{gamma}, C={C}', fontsize=18)

    print(f'Anzahl Support Vectors: {svm.n_support_}')


In [ ]:
pltSvmRbf(C=1, gamma=0.0001)

In [ ]:
pltSvmRbf(C=100, gamma=0.0001)

In [ ]:
pltSvmRbf(C=1, gamma=1)

In [ ]:
pltSvmRbf(C=100, gamma=0.1)


Wie man an den Beispielen sieht, ist eine blinde Suche nach den richtigen Hyperparametern das Stochern im Heuhaufen.

Bei der Suche muss man gezielt vorgehen, etwa mit einer (logarithmischen) Grid-Suche:

In [ ]:
def anzahlSupportVectors(C, gamma):
    svm = SVC(kernel='rbf', random_state=0, gamma=gamma, C=C)
    svm.fit(X_xor, y_xor)
    return np.sum(svm.n_support_)

C_hyper = [10.0**n for n in np.arange(0, 6)]
gamma_hyper = [10.0**n for n in np.arange(-5, 3)]
X_bins = np.asarray([[C,gamma] for C in C_hyper for gamma in gamma_hyper])
y_bins = np.asarray([anzahlSupportVectors(C,gamma) for [C,gamma] in X_bins])

fi, ax = plt.subplots(1, figsize=(10,10))
ax.set_xlabel('C')
ax.set_ylabel('gamma')
ax.set_xscale('log')
ax.set_yscale('log')
ax.scatter(X_bins[:,0], X_bins[:,1], s=(10*y_bins));

y_min = np.argmin(y_bins)
[C_min, gamma_min] = X_bins[y_min]
print(f'Minimale Anzahl an Support Vektoren bei [C, gamma] = [{C_min}, {gamma_min}] ist {y_bins[y_min]}')

Weil diese Suche nach Hyper-Parametern ein Standard Schritt in der Entwicklung von Machine-Learning Modellen ist, bietet Scikit-Learn eine entsprechende Methode an:

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel':np.asarray(['rbf']), 'C':C_hyper, 'gamma':gamma_hyper}

grid = GridSearchCV(SVC(), parameters)
grid.fit(X_xor, y_xor)

print("Die besten Parameter sind %s mit einem Score von %0.2f" % (grid.best_params_, grid.best_score_))

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

@interact(C=C_hyper, gamma=gamma_hyper)
def svc(C=C_min, gamma=gamma_min):
  # Create a SVC classifier using an RBF kernel
  svm = SVC(kernel='rbf', random_state=0, gamma=gamma, C=C)
  # Train the classifier
  svm.fit(X_xor, y_xor)

  # Visualize the decision boundaries
  ax = plot_decision_surface(X_xor, y_xor, svm)
  plot_hyperplane_margin(X_xor, y_xor, svm, ax);
  print(f'Anzahl Support Vectors: {svm.n_support_}')


In [ ]:
pltSvmRbf(C=100000, gamma=0.01)
